In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #type: ignore
from tensorflow.keras.applications import ResNet50 #type: ignore
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout#type: ignore
from tensorflow.keras.models import Model #type: ignore
from tensorflow.keras.optimizers import Adam #type: ignore
from tensorflow.keras.preprocessing import image #type: ignore
import splitfolders
import os, random, shutil
from pathlib import Path
import json
import matplotlib.pyplot as plt

In [3]:
import numpy
print(numpy.__version__)
#print(pandas.__version__)



1.26.4


In [ ]:
df=pd.read_csv('dataset_stats.csv')

df.describe()


In [ ]:
for name, count in zip(df['class'], df['image_count']):
    print(name,count)


In [ ]:
df[['avg_width','avg_height']].plot(kind='box')


In [ ]:

src = Path("sneakers-dataset")   # each subfolder = class
dst = Path("splits")               # keep OUTSIDE src
allowed = {".jpg", ".jpeg", ".png", ".bmp", ".webp", ".jfif"}
random.seed(42)

# clean/create targets
for part in ("train","val","test"):
    (dst/part).mkdir(parents=True, exist_ok=True)

total = 0
for cls_dir in sorted([d for d in src.iterdir() if d.is_dir()]):
    files = [p for p in cls_dir.iterdir() if p.is_file() and p.suffix.lower() in allowed]
    if not files:
        print(f"[warn] no images in: {cls_dir}")
        continue

    random.shuffle(files)
    n = len(files); n_train = int(0.8*n); n_val = int(0.1*n)

    splits = {
        "train": files[:n_train],
        "val":   files[n_train:n_train+n_val],
        "test":  files[n_train+n_val:],
    }

    for part, flist in splits.items():
        out = dst/part/cls_dir.name
        out.mkdir(parents=True, exist_ok=True)
        for f in flist:
            shutil.copy2(f, out/f.name)

    total += n
    print(f"{cls_dir.name}: {n} -> train {n_train}, val {n_val}, test {n - n_train - n_val}")

print("Done. Total images:", total)



In [ ]:


train_gen = ImageDataGenerator(
    rescale=1./255,                     # [snippet] scale pixels to [0,1]

    rotation_range=20,                  # [snippet] rotate within ±20°
    # rotation_range=0                  # disable rotation

    width_shift_range=0.10,             # [snippet] horizontal shift up to ±10%
    height_shift_range=0.10,            # [snippet] vertical shift up to ±10%
    # width_shift_range=0.0             # disable width shift
    # height_shift_range=0.0            # disable height shift

    shear_range=0.15,                   # [snippet] projective skew
    # shear_range=0.0                   # disable shear

    zoom_range=0.20,                    # [snippet] zoom in/out up to 20%
    # zoom_range=[1.0, 1.2]             # only zoom-in
    # zoom_range=[0.8, 1.0]             # only zoom-out

    brightness_range=[0.7, 1.3],        # [snippet] random brightness
    # brightness_range=None             # disable brightness jitter

    channel_shift_range=30,             # [snippet] random RGB shift (color cast)
    # channel_shift_range=0             # disable color shift

    horizontal_flip=True,               # [snippet] left-right flip
    # horizontal_flip=False             # disable flip

    fill_mode='nearest'                 # [snippet] fill gaps after transforms
    # fill_mode='reflect'               # alternative
    # fill_mode='constant'              # with cval=0 (black)
    # cval=128                          # used if fill_mode='constant'
)


val_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_gen.flow_from_directory(
    "./splits/train", target_size=(224,224),
    batch_size=32, shuffle=True, class_mode="categorical", seed=42
)
val = val_gen.flow_from_directory(
    "./splits/val", target_size=(224,224),
    batch_size=32, shuffle=False, class_mode="categorical"
)


In [ ]:
import matplotlib.pyplot as plt, numpy as np

x, y = next(train)              # one batch
for i in range(9):
    plt.figure()
    plt.imshow(np.clip(x[i], 0, 1))
    plt.axis("off")


In [ ]:
print(tf.config.list_physical_devices('GPU'))


In [ ]:
base = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

# freeze most layers
for layer in base.layers[:-10]:
    layer.trainable = False

In [ ]:
x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
x = Dense(256, activation="relu")(x)
out = Dense(train.num_classes, activation="softmax")(x)

model = Model(inputs=base.input, outputs=out)
model.compile(
    optimizer=Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
history = model.fit(
    train,
    validation_data=val,
    epochs=10
)

val_loss, val_acc = model.evaluate(val)
print(f"Validation accuracy: {val_acc:.3f}")


In [ ]:
# load trained model
model = tf.keras.models.load_model("best_resnet50_sneakers.h5")

# load label mapping
with open("class_indices.json") as f:
    class_indices = json.load(f)
idx_to_class = {v: k for k, v in class_indices.items()}


In [ ]:
img_path = "./jordan12.jpeg"
img = image.load_img(img_path, target_size=(224,224))   # same size as training
x = image.img_to_array(img)
x = x / 255.0   

plt.imshow(x)
plt.axis("off")
plt.show()                                        # same rescale
x = np.expand_dims(x, axis=0)

pred = model.predict(x)
i = np.argmax(pred[0])
print("Predicted class:", idx_to_class[i])
print("Confidence:", pred[0][i])


In [ ]:
from ultralytics import YOLO

# community model with 'shoe' class (example, replace if removed)
model_det = YOLO("yolov8s.pt")
results = model_det("./samba.jpg", save=True, save_crop=True, project="./", name="footwear_crops")
